# CP SAT fun

AKA misusing the CP SAT solver for mischevious fun. 

In [3]:
from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print("%s=%i" % (v, self.Value(v)), end=" ")
        print()

    def solution_count(self):
        return self.__solution_count


## Division with decimal numbers

CP-SAT is an integer programming solver. Thus, it only stores integer variables. 

But when you divide variables, you sometimes need more precision than an integer. 

Simply multiply the variable by a big number to get a rounded down approximation. 

In [587]:
FLOAT_APPROX_PRECISION = 100

In [588]:
model = cp_model.CpModel()

numerator = model.NewIntVar(0, 100, "numerator")
denominator = model.NewIntVar(1, 100, "denominator")
result = model.NewIntVar(0, 100, "result")

# We want to compute 10/30
model.Add(numerator == 10)
model.Add(denominator == 30)
# Add a divison equality and multiply numerator by 100
model.AddDivisionEquality(result, numerator * FLOAT_APPROX_PRECISION, denominator)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is approximately: {solver.Value(result) / FLOAT_APPROX_PRECISION}")


Solution is approximately: 0.33


## Constraints with decimal numbers

Let's say you want the result variable to be more than half of the variable x. This will fail.

In [589]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100, "result")

# Assume x == 30
model.Add(numerator == 30)
# This will fail
model.Add(result >= 0.5 * x)

TypeError: Not an integer: -0.5

But you can rewrite this inequality, to say that the variable result should be at least twice as big as the variable x.

In this casee, the constraint is perfectly valid.

In [590]:
model.Add(2 * result >= x)

More generally, a quick and dirty way to handle constraints with decimal numbers is to multiply both sides by a big number and round everything. 

In [591]:
# This will work well enough in most cases
model.Add(FLOAT_APPROX_PRECISION*result >= round(FLOAT_APPROX_PRECISION * 0.5) * x)

## Only enforce a constraint when a condition is verified

A great thing in CP SAT is being able to enable constraint only when some bool is true. 

However, this method doesn't work out of the box with linear expressions.

In [592]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")

# We want to have result == x only if x >= 0
# Let's suppose x=10
model.Add(x == 10)
# The following will fail 
model.Add(result == x).OnlyEnforceIf(x >= 0)

TypeError: 'BoundedLinearExpression' object is not iterable

A trick is to use an intermediate boolean variable, with correct constraints.

The boolean variable will be true when the condition is enforced, and false otherwise. 

Here, we implement this for the condition "var is non zero", i.e. `var > 0`. We use a Big M to make this.

In [593]:
BIG_M = 1_000 # Big M should be larger than x largest absolute value

def create_boolean_is_positive(model:cp_model.CpModel, var: cp_model.IntVar):
    """Create a bool variable such that 
    If var >= 0 then bool = 1
    If var  < 0 then bool = 0 
    """
    boolean_var = model.NewBoolVar(name=var.Name() + "_is_positive")

    # Bool are casted to 0 if False and 1 if True, so you can do some operations with them

    # If var > 0, then this is true only if bool = 1
    # If var <= 0, then this is always true
    model.Add(BIG_M*boolean_var >= var)
    # If var > 0, then this is always true
    # If var <= 0, then this is true only if bool = 0
    model.Add(BIG_M*(boolean_var - 1) <= var)

    # To handle the case var == 0, we specifiy that we want boolean_var to be = 1 this way
    model.Add(boolean_var > var).OnlyEnforceIf(boolean_var.Not())

    return boolean_var

Let's try this function. If x is nonzero positive, we'd like to see the value of x as a result. 

In [594]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")

# We want to have result == x only if x >= 0
# We create the intermediate variable 
x_is_positive = create_boolean_is_positive(model, x)

# Let's suppose x=10
model.Add(x == 10)
# The following will work
model.Add(result == x).OnlyEnforceIf(x_is_positive)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(result)}")


Solution is: 10


Now if x is nonzero negative, we set the result to be 42. 

In [595]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")
x_is_positive = create_boolean_is_positive(x)

# Let's suppose x=-10
model.Add(x == -10)
model.Add(result == x).OnlyEnforceIf(x_is_positive)
# And return 42 if x is not positive
model.Add(result == 42).OnlyEnforceIf(x_is_positive.Not())


# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(result)}")

Solution is: 42


Let's also inspect the value of the bool if x=0. In the function we decided for the bool to be true. This is indeed what we see. 

In [596]:
model = cp_model.CpModel()

x = model.NewIntVar(-100, 100, "x")
result = model.NewIntVar(0, 100, "result")
x_is_positive = create_boolean_is_positive(x)

# Let's test the case x==0 
model.Add(x == 0)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is: {solver.Value(x_is_positive)}")

Solution is: 1


## Multiplication of three or more terms with intermediate variables

The solver works with a multiplication equality constraint of 2 variables. 

In [597]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(1, 100, "y")
result = model.NewIntVar(0, 100 * 100, "result")

# We want to compute 22*11
model.Add(x == 22)
model.Add(y == 11)
model.AddMultiplicationEquality(result, [x, y])

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Solution is: {solver.Value(result)}")


Solution is: 242


But with more than 2 terms, the solver will fail. This is despite the docs saying us that it should work. 

In [598]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(0, 100, "y")
z = model.NewIntVar(0, 100, "z")
result = model.NewIntVar(0, 100 * 100 * 100, "result")

# We want to compute 22*11*33
model.Add(x == 22)
model.Add(y == 11)
model.Add(z == 33)
model.AddMultiplicationEquality(result, [x, y, z])

# The solver fails
solver = cp_model.CpSolver()
status = solver.Solve(model)
print(f"Status = {solver.StatusName(status)}")


Status = MODEL_INVALID


A trick is to use an intermediate variable. 

In [599]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
y = model.NewIntVar(0, 100, "y")
intermediate = model.NewIntVar(0, 100 * 100, "x*y")
z = model.NewIntVar(0, 100, "z")
result = model.NewIntVar(0, 100 * 100 * 100, "result")

# Let's say we want to compute 22*11*33
# First we compute x*y and store the result in an intermediate variable
model.Add(x == 22)
model.Add(y == 11)
model.AddMultiplicationEquality(intermediate, [x, y])
# And then we multiply this intermediate variable and get the final result
model.Add(z == 33)
model.AddMultiplicationEquality(result, [intermediate, z])

# The solver is happy now
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solution is: {solver.Value(result)}")

Status = OPTIMAL
Solution is: 7986


We can generalize this idea, and implement this recursive function. 

In [600]:
from typing import List


def add_multiplication_constraint(
    model: cp_model.CpModel, target: cp_model.IntVar, variables: List[cp_model.IntVar],
):
    if len(variables) <= 2:
        # If less than 2 variables, we can add a normal inequality constraint
        model.AddMultiplicationEquality(target, variables)
        return
    else:
        last_variable = variables.pop()
        before_last_variable = variables.pop()
        # Use their bounds to define domain of the intermediate variable
        # You may need additional logic here to account for variable domain bounds
        ub = max(
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[0],
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[0],
        )
        lb = min(
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[1],
            last_variable.Proto().domain[0] * before_last_variable.Proto().domain[0],
            last_variable.Proto().domain[1] * before_last_variable.Proto().domain[0],
        )
        # Create an intermediate variable
        intermediate = model.NewIntVar(
            lb=lb, ub=ub, name=f"{before_last_variable.Name()}*{last_variable.Name()}"
        )
        model.AddMultiplicationEquality(
            intermediate, [before_last_variable, last_variable]
        )
        # Recursion
        add_multiplication_constraint(model, target, variables + [intermediate])


Let's test it with the product of 10 variables with random values. 

In [601]:
from random import randint 

model = cp_model.CpModel()

n_variables = 10
many_variables = [model.NewIntVar(-20, 20, f"{x}_{i}") for i in range(n_variables)]
result = model.NewIntVar(- 20 ** n_variables, 20 ** n_variables, "result")

# Set the var to be equal to random int between 1 and 10
real_product = 1
for i, var in enumerate(many_variables):
    random_value = randint(-20, 20)
    if random_value == 0:
        random_value += 1
    real_product *= random_value
    print(f"Set {x}_{i}={random_value}")
    model.Add(var == random_value)

add_multiplication_constraint(model, result, many_variables)

# The solver is happy now
solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solution found by the solver is: {solver.Value(result)}")
print(f"Real solution is: {real_product}")

Set x_0=-19
Set x_1=17
Set x_2=-12
Set x_3=11
Set x_4=-1
Set x_5=11
Set x_6=-13
Set x_7=-12
Set x_8=-10
Set x_9=4
Status = OPTIMAL
Solution found by the solver is: 2926535040
Real solution is: 2926535040


This works ! But beware, as integers have limits, and you can reach them when you multiply many big numbers together.
 
I found the limit to be $2^{63} \approx 9,2 \times 10^{18}$. 

In [602]:
# No problem
big_int = model.NewIntVar(- 2**63 + 1, 2**63 -1, "big_int")

In [603]:
# Big trouble !!!
big_int = model.NewIntVar(0, 2**63, "big_int")

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. ortools.util.python.sorted_interval_list.Domain(arg0: int, arg1: int)

Invoked with: 0, 9223372036854775808

## Exponential, polynoms, and Taylor expansions

CP SAT supports many integer operations. But you can't compose an IntVar by any function. For example, you can't do exp(var).

In [604]:
from math import exp

model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100 * 100, "result")

# Let's say we want result == exp(x)
# This will fail (rightfully)
model.Add(result == exp(x))

TypeError: must be real number, not IntVar

### Taylor series approximation for exp at integer values

But... What we can do is compute products of variables. 

Thus, we can set constraints on polynoms.

And using [Taylor's theorem](https://en.wikipedia.org/wiki/Taylor%27s_theorem), we can approximate functions by polynoms using their n-th derivatives, for their values close to zero. For example, 

$$e^{x} = \sum_{k=0}^{\infty} \frac{x^k}{k!} \approx 1 + x + \frac{1}{2}x^2 + \frac{1}{6}x^3 + \frac{1}{24}x^4 + \dots$$

$$\ln(1+x) \approx x - \frac{1}{2}x^2 + \frac{1}{3}x^3 - \frac{1}{4}x^4 + \dots $$

Note that this can be used to approximate many other functions, but only close to zero. 

So, let's create polynoms.

In [605]:
def create_polynom(model: cp_model.CpModel, var: cp_model.IntVar, coefs: List[float], float_precision=FLOAT_APPROX_PRECISION, verbose=True):
    degree = len(coefs)

    # Approximate all coef values by mutliplying them by a big number and rounding them
    coefs = [round(float_precision * coef) for coef in coefs]

    polynom_value = 0
    polynom_value_ub = 0
    polynom_value_lb = 0
    for deg in range(degree):
        # Create the coefficient value
        if deg == 0:
            polynom_value += coefs[deg]
            polynom_value_lb += coefs[deg]
            polynom_value_ub += coefs[deg]
        elif deg == 1:
            polynom_value_lb += var.Proto().domain[0] * coefs[deg]
            polynom_value_ub += var.Proto().domain[1] * coefs[deg]

            polynom_value += coefs[deg] * var
        else:
            lb = var.Proto().domain[0] ** deg
            ub = var.Proto().domain[1] ** deg

            if (deg % 2) == 0:
                if var.Proto().domain[0] < 0:
                    lb = - lb
                if var.Proto().domain[1] < 0:
                    ub = - ub
            
            lb = coefs[deg] * lb
            ub = coefs[deg] * ub      

            polynom_value_lb += lb
            polynom_value_ub += ub 
            
            target = model.NewIntVar(lb=lb, ub=ub, name=f"{var.Name()}**{deg}")
            add_multiplication_constraint(model, target, [var]*deg)
            polynom_value += coefs[deg]*target
    
    if verbose:
        print("Polynom", polynom_value)

    polynom_var = model.NewIntVar(polynom_value_lb, polynom_value_ub, name=f"{var.Name()}_polynom")
    model.Add(polynom_var == polynom_value)
    return polynom_var

This previous function creates an intvar equals to the polynom value at x, with the given coefficients rounded. 

Here is the polynom associated to the taylor series expansion of exp.

In [606]:
from math import factorial

float_precision = 10_000
polynom_degree_approx = 5
value_of_x = 1

model = cp_model.CpModel()

x = model.NewIntVar(0, 10, "x")
exp_of_x = create_polynom(model, x, coefs=[1 / factorial(k) for k in range(polynom_degree_approx)], float_precision=float_precision)

model.Add(x == value_of_x)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x) / float_precision}")
print(f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }")
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((10000 * x) + 10000) + (5000 * x**2)) + (1667 * x**3)) + (417 * x**4))
Status = OPTIMAL
Solver value approximation of exp(1) is: 2.7084
Real approximation is: 2.708333333333333
Real value is: 2.718281828459045


This can also work for negative value of exp, although we need to crank up the precision and degrees. 

In [607]:
float_precision = 10_000_000
polynom_degree_approx = 11
value_of_x = -3

model = cp_model.CpModel()

x = model.NewIntVar(-10, 10, "x")
exp_of_x = create_polynom(model, x, coefs=[1 / factorial(k) for k in range(polynom_degree_approx)], float_precision=float_precision)

model.Add(x == value_of_x)

solver = cp_model.CpSolver()
status = solver.Solve(model)


print(f"Status = {solver.StatusName(status)}")
print(f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x) / float_precision}")
print(f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }")
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((((((((10000000 * x) + 10000000) + (5000000 * x**2)) + (1666667 * x**3)) + (416667 * x**4)) + (83333 * x**5)) + (13889 * x**6)) + (1984 * x**7)) + (248 * x**8)) + (28 * x**9)) + (3 * x**10))
Status = OPTIMAL
Solver value approximation of exp(-3) is: 0.0539323
Real approximation is: 0.05332589285714289
Real value is: 0.049787068367863944


### Taylor series approximation for exp at decimal values

Now, this approximation is cool and all. But the main use of Taylor approximation is for values close to zero. Indeed, that's where the approximation is the best. Right now, we only have approximation of integer values. 

An intuitive way to do that would be to multiply the value of x by a big number, like we did for the division. Just use `67` instead of `0.67`.

But with the current implementation, this leads to big numerical errors. For example, `67**4 + 67**2 = 20 151 121` but `0.67**4 + 0.67**2 = 0.65041121`. 

We solve this by approximating every power individually, i.e. `67**4 / 100**3 + 67**2 / 100**1 = 65`, which is much closer to what we need.

In [608]:
67**4 / 100**3 + 67**2 / 100**1

65.041121

In [609]:
def create_polynom_decimal(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    coefs: List[float],
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_coef=FLOAT_APPROX_PRECISION,
    verbose=True,
):
    """This polynom accepts as an input a decimal var upscaled by float_precision_var."""
    degree = len(coefs)

    # Approximate all coef values by mutliplying them by a big number and rounding them
    coefs = [round(float_precision_coef * coef) for coef in coefs]

    polynom_value = 0
    polynom_value_ub = 0
    polynom_value_lb = 0
    for deg in range(degree):
        # Create the coefficient value
        if deg == 0:
            polynom_value += coefs[deg] * float_precision_var
            polynom_value_lb += coefs[deg] * float_precision_var
            polynom_value_ub += coefs[deg] * float_precision_var
        elif deg == 1:
            polynom_value_lb += var.Proto().domain[0] * coefs[deg]
            polynom_value_ub += var.Proto().domain[1] * coefs[deg]

            polynom_value += coefs[deg] * var 
        else:
            # Bounds logic
            lb_no_coef = var.Proto().domain[0] ** deg
            ub_no_coef = var.Proto().domain[1] ** deg

            if (deg % 2) == 0:
                if var.Proto().domain[0] < 0:
                    lb_no_coef = -lb_no_coef
                if var.Proto().domain[1] < 0:
                    ub_no_coef = -ub_no_coef

            lb = coefs[deg] * lb_no_coef
            ub = coefs[deg] * ub_no_coef

            polynom_value_lb += lb
            polynom_value_ub += ub

            # Compute (x**n)
            target = model.NewIntVar(lb=lb_no_coef, ub=ub_no_coef, name=f"{var.Name()}**{deg}")
            add_multiplication_constraint(model, target, [var] * deg)
            # Then compute (a * x**n)
            target_times_coef = model.NewIntVar(
                lb=lb, ub=ub, name=f"{coefs[deg]}*{var.Name()}**{deg}"
            )
            model.Add(target_times_coef == target * coefs[deg])
            # Downscale (a * x**n) to the float_precision_var range
            target_divided_by_approx = model.NewIntVar(
                lb=lb, ub=ub, name=f"{coefs[deg]}*{var.Name()}**{deg} / ({float_precision_var**(deg-1)})"
            )
            model.AddDivisionEquality(
                target_divided_by_approx, target_times_coef, float_precision_var**(deg-1)
            )

            polynom_value += target_divided_by_approx

    if verbose:
        print("Polynom", polynom_value)

    polynom_var = model.NewIntVar(
        polynom_value_lb, polynom_value_ub, name=f"{var.Name()}_polynom"
    )
    model.Add(polynom_var == polynom_value)
    return polynom_var


And now we can evaluate polynoms at decimal values, and consequently compute Taylor expansion closer to zero. 

Note that since we are evaluating closer to zero, we can downgrade the Taylor expansion precision and still get good results


In [610]:
float_precision_var = 100
float_precision_coefs = 1_000
polynom_degree_approx = 5
value_of_x = 0.69

model = cp_model.CpModel()

x = model.NewIntVar(-10 * float_precision_var, 10 * float_precision_var, "x")
exp_of_x = create_polynom_decimal(
    model,
    x,
    coefs=[1 / factorial(k) for k in range(polynom_degree_approx)],
    float_precision_var=float_precision_var,
    float_precision_coef=float_precision_coefs,
    verbose=True,
)

model.Add(x == round(float_precision_var * value_of_x))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

print(
    f"Solver value approximation of exp({value_of_x}) is: {solver.Value(exp_of_x)/ float_precision_coefs / float_precision_var}"
)
print(
    f"Real approximation is: { sum([value_of_x**k / factorial(k) for k in range(polynom_degree_approx)]) }"
)
print(f"Real value is: {exp(value_of_x)}")


Polynom (((((1000 * x) + 100000) + 500*x**2 / (100)) + 167*x**3 / (10000)) + 42*x**4 / (1000000))
Status = OPTIMAL
Solver value approximation of exp(0.69) is: 1.99243
Real approximation is: 1.99224613375
Real value is: 1.9937155332430823


### Precompute the function values

Taylor series are cool, but a bit of an overkill. Especially at this coarse level of precision (`10E-2`) and for a function with such well-known values. 

Instead of approximating exp, we could simply precompute its values, and use some kind of hashmap to refer to it.

In [618]:
import numpy as np


def create_boolean_is_equal_to(model:cp_model.CpModel, var: cp_model.IntVar, value: int):
    """Create a bool variable such that
    If var == value then bool = 1
    Else then bool = 0
    """
    boolean_var = model.NewBoolVar(name=f"{var.Name()}_is_equal_to_{value}")
    model.Add(value * boolean_var == var).OnlyEnforceIf(boolean_var)
    model.Add(value != var).OnlyEnforceIf(boolean_var.Not())

    return boolean_var


def lookup_table_exp_of_x(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_exp=FLOAT_APPROX_PRECISION,
):
    lb = var.Proto().domain[0]
    ub = var.Proto().domain[1]
    x_to_exp_x = {
        x: round(exp(x / float_precision_var) * float_precision_exp)
        for x in np.arange(lb, ub + 1)
    }
    exp_of_var = model.NewIntVar(
        min(x_to_exp_x.values()), max(x_to_exp_x.values()), f"exp_{var.Name()}"
    )

    # This is how we implement a kind of lookup table
    for x_value, exp_value in x_to_exp_x.items():
        var_is_equal_to_x = create_boolean_is_equal_to(model, var, x_value)
        model.Add(exp_of_var == exp_value).OnlyEnforceIf(var_is_equal_to_x)
    
    return exp_of_var

In [619]:
float_precision_var = 100
float_precision_exp = 10_000
value_of_x = 0.69

model = cp_model.CpModel()

x = model.NewIntVar(-10 * float_precision_var, 10 * float_precision_var, "x")
exp_of_x = lookup_table_exp_of_x(
    model,
    x,
    float_precision_var=float_precision_var,
    float_precision_exp=float_precision_exp,
)

model.Add(x == round(float_precision_var * value_of_x))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

print(
    f"Solver value of exp({value_of_x}) is: {solver.Value(exp_of_x)/ float_precision_exp}"
)
print(f"Real value is: {exp(value_of_x)}")


Status = OPTIMAL
Solver value of exp(0.69) is: 1.9937
Real value is: 1.9937155332430823


/tmp/ipykernel_4537/1959322448.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  model.Add(value != var).OnlyEnforceIf(boolean_var.Not())
